# Creación de DataFrames
Crearemos objetos `DataFrame` a partir de otras estructuras de datos en Python, leyendo un fichero CSV, y consultando una base de datos.

## Acerca de los datos
En este cuaderno trabajaremos con datos de terremotos del 18 de septiembre de 2018 al 13 de octubre de 2018 (obtenidos del US Geological Survey (USGS) usando la [USGS API](https://earthquake.usgs.gov/fdsnws/event/1/))

## Importaciones

In [2]:
import datetime as dt
import numpy as np
import pandas as pd

## Creación de un objeto `Series`

In [ ]:
np.random.seed(0) # fijar la semilla para que el resultado sea reproducible
pd.Series(np.random.rand(5), name='random')

## Crear un objeto `DataFrame` a partir de un objeto `Series`
Utilice el método `to_frame()`:

In [ ]:
pd.Series(np.linspace(0, 10, num=5)).to_frame()

## Creación de un `DataFrame` a partir de Estructuras de Datos Python
### A partir de un diccionario de estructuras tipo lista
Los valores del diccionario pueden ser listas, matrices NumPy, etc. siempre que tengan longitud (los generadores no tienen longitud por lo que no podemos usarlos aquí):

In [ ]:
np.random.seed(0) # fijar la semilla para que el resultado sea reproducible
pd.DataFrame(
    {
        'random': np.random.rand(5),
        'texto': ['caliente', 'tibio', 'fresco', 'frio', None],
        'verdad': [np.random.choice([True, False]) for _ in range(5)]
    }, 
    index=pd.date_range(
        end=dt.date(2019, 4, 21),
        freq='1D',
        periods=5, 
        name='date'
    )
)

### De una lista de diccionarios

In [ ]:
pd.DataFrame([
    {'mag': 5.2, 'place': 'California'},
    {'mag': 1.2, 'place': 'Alaska'},
    {'mag': 0.2, 'place': 'California'},
])

### De una lista de tuplas

In [ ]:
list_of_tuples = [(n, n**2, n**3) for n in range(5)]
list_of_tuples

In [ ]:
pd.DataFrame(
    list_of_tuples, 
    columns=['n', 'n_cuadrado', 'n_cubo']
)

### De un array NumPy

In [ ]:
pd.DataFrame(
    np.array([
        [0, 0, 0],
        [1, 1, 1],
        [2, 4, 8],
        [3, 9, 27],
        [4, 16, 64]
    ]), columns=['n', 'n_cuadrado', 'n_cubo']
)

## Creación de un objeto `DataFrame` a partir del contenido de un fichero CSV

### Buscando información sobre el fichero antes de leerlo
Antes de intentar leer un fichero, podemos usar la línea de comandos para ver información importante sobre el fichero que puede determinar cómo lo leemos. Podemos ejecutar código de línea de comandos desde Jupyter Notebooks (gracias a IPython) usando `!` antes del código.

#### Número de líneas (recuento de filas)
Por ejemplo, podemos averiguar cuántas líneas hay en el archivo utilizando la utilidad `wc` (recuento de palabras) y contando las líneas del archivo (`-l`). El fichero tiene 9.333 líneas:

In [3]:
# !wc -l data/earthquakes.csv

!find /c /v "" data\earthquakes.csv


---------- DATA\EARTHQUAKES.CSV: 9333


**Usuarios de Windows**: si lo anterior no te funciona (depende de tu configuración), utiliza esto en su lugar:

```python
!find /c /v "" data\earthquakes.csv
```



#### Tamaño del archivo
Podemos encontrar el tamaño del archivo usando `ls` para listar los archivos en el directorio `data`, y pasando las banderas `-lh` para incluir el tamaño del archivo en formato legible por humanos. Luego usamos `grep` para encontrar el fichero en cuestión. Ten en cuenta que `|` pasa el resultado de `ls` a `grep`. La utilidad `grep` se utiliza para encontrar elementos que coincidan con patrones.

Esto nos dice que el archivo tiene 3,4 MB:

In [4]:
# !ls -lh data | grep earthquakes.csv

!dir data | findstr "earthquakes.csv"

27/10/2023  18:42         3.524.989 earthquakes.csv



**Usuarios de Windows**: si lo anterior no te funciona (depende de tu configuración), utiliza esto en su lugar:

``python
!dir data | findstr "earthquakes.csv"
```

Incluso podemos capturar el resultado de un comando y utilizarlo en nuestro código Python:

In [5]:
files = !dir data
[file for file in files if 'earthquake' in file]

['27/10/2023  18:42         3.524.989 earthquakes.csv']

**Usuarios de Windows**: si lo anterior no te funciona (depende de tu configuración), utiliza esto en su lugar:

```python
files = !dir data
[file for file in files if 'earthquake' in file]
```


#### Examinar algunas filas
Podemos utilizar `head` para examinar las `n` primeras filas del fichero. Con la bandera `-n`, podemos especificar cuántas. Esto nos muestra que la primera fila del fichero contiene cabeceras y que están separadas por comas (el hecho de que la extensión del fichero sea `.csv` no significa que contenga valores separados por comas):

In [6]:
# !head -n 2 data/earthquakes.csv

n = 2
with open('data/earthquakes.csv', 'r') as file:
    for _ in range(n):
        print(file.readline(), end='\r')

alert,cdi,code,detail,dmin,felt,gap,ids,mag,magType,mmi,net,nst,place,rms,sig,sources,status,time,title,tsunami,type,types,tz,updated,url
,,37389218,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci37389218&format=geojson,0.008693,,85.0,",ci37389218,",1.35,ml,,ci,26.0,"9km NE of Aguanga, CA",0.19,28,",ci,",automatic,1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475395144,https://earthquake.usgs.gov/earthquakes/eventpage/ci37389218


**Usuarios de Windows**: si lo anterior no te funciona (depende de tu configuración), utiliza esto en su lugar:

```python
n = 2
with open('data/earthquakes.csv', 'r') as file:
    for _ in range(n):
        print(file.readline(), end='\r')
```


Igual que `head` da las filas de arriba, `tail` da las filas de abajo. Esto puede ayudarnos a comprobar que no hay datos extraños en la parte inferior del campo, como quizás algunos metadatos sobre los campos que en realidad no forman parte del conjunto de datos:

In [ ]:
!tail -n 1 data/earthquakes.csv

**Usuarios de Windows**: si lo anterior no te funciona (depende de tu configuración), utiliza esto en su lugar:

```python
import os

with open('data/earthquakes.csv', 'rb') as file:
    file.seek(0, os.SEEK_END)
    while file.read(1) != b'\n':
        file.seek(-2, os.SEEK_CUR)
    print(file.readline().decode())
```

*Nota Para inspeccionar más de una fila desde el final del fichero, tendrás que usar esto en su lugar, lo que requiere leer todo el fichero:

```python
n = 2
with open('data/earthquakes.csv', 'r') as file:
    print('\r'.join(file.readlines()[-n:]))
```

#### Recuento de columnas
Podemos utilizar `awk` para encontrar el recuento de columnas. Se trata de una utilidad para escanear y procesar patrones. La bandera `-F` nos permite especificar el delimitador (coma, en este caso). Luego especificamos qué hacer para cada registro del fichero. Elegimos imprimir `NF`, que es una variable predefinida cuyo valor es el número de campos del registro actual. Aquí, decimos `exit` para que imprimamos el número de campos (columnas, aquí) en la primera fila del fichero, y luego nos detenemos.

Esto nos dice que tenemos 26 columnas de datos:

In [ ]:
!awk -F',' '{print NF; exit}' data/earthquakes.csv


**Usuarios de Windows**: si lo anterior o lo siguiente no te funciona (depende de tu configuración), utiliza esto en su lugar:

```python
with open('data/earthquakes.csv', 'r') as file:
    print(len(file.readline().split(',')))
```


Como sabemos que la primera línea del fichero tiene cabeceras, y el fichero está separado por comas, también podemos contar las columnas usando `head` para obtener las cabeceras y analizarlas en Python:

In [ ]:
headers = !head -n 1 data/earthquakes.csv
len(headers[0].split(','))

**Usuarios de Windows**: si tiene que utilizar las alternativas anteriores, considere probar [Cygwin](https://www.cygwin.com) o [Windows Subsystem for Linux (WSL)](https://docs.microsoft.com/en-us/windows/wsl/about).

### Lectura del fichero
Nuestro fichero es de pequeño tamaño, tiene cabeceras en la primera fila, y está separado por comas, por lo que no necesitamos proporcionar ningún argumento adicional para leer el fichero con `pd.read_csv()`, pero asegúrate de revisar la [documentación](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) para posibles argumentos:

In [7]:
df = pd.read_csv('data/earthquakes.csv')

Pandas es normalmente muy bueno en averiguar qué opciones utilizar basándose en los datos de entrada, por lo que a menudo no necesitaremos añadir argumentos a la llamada; sin embargo, hay muchas opciones disponibles si las necesitamos, algunas de las cuales incluyen las siguientes:

| Parámetro | Propósito |
| --- | --- |
| `sep` | Especifica el delimitador |
| `header` | Número de fila donde se encuentran los nombres de las columnas; la opción por defecto hace que `pandas` deduzca si están presentes |
| `names` | Lista de nombres de columnas para utilizar como cabecera |
| `index_col` | Columna que se utilizará como índice |
| `usecols` | Especifica qué columnas leer |
| `dtype` | Especifica los tipos de datos de las columnas | 
| `converters` | Especifica funciones para convertir datos en determinadas columnas |
| `skiprows` | Filas a saltar |
| `nrows` | Número de filas a leer a la vez (combinar con `skiprows` para leer un fichero bit a bit) |
| `parse_dates` | Convertir automáticamente las columnas que contienen fechas en objetos datetime |
| `chunksize` | Para leer el archivo en trozos |
| `compression` | Para leer archivos comprimidos sin extraerlos previamente |
| `encoding` | Especifica la codificación del archivo |

## Escribir un objeto `DataFrame` en un fichero CSV
Ten en cuenta que el índice de `df` son sólo números de fila, por lo que no queremos conservarlo. Por lo tanto, pasamos `index=False` al método `to_csv()`:

In [8]:
df.to_csv('output.csv', index=False)

## Escribir un objeto `DataFrame` en una base de datos
Fíjate en el parámetro `if_exists`. Por defecto, te dará un error si intentas escribir una tabla que ya existe. Aquí, no nos importa si se sobrescribe. Por último, si estamos interesados en añadir nuevas filas, le damos el valor `'append'`.

In [10]:
!pip install SQLAlchemy

In [11]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('sqlite:///data/quakes.db')

pd.read_csv('data/tsunamis.csv').to_sql(
    'tsunamis', engine, index=False, if_exists='replace'
)


61

## Crear un objeto `DataFrame` consultando una base de datos
Usando una base de datos SQLAlchemy

In [12]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('sqlite:///data/quakes.db')

tsunamis = pd.read_sql('SELECT * FROM tsunamis', engine)

tsunamis.head()


,alert,type,title,place,magType,mag,time
0,None,earthquake,"M 5.0 - 165km NNW of Flying Fish Cove, Christm...","165km NNW of Flying Fish Cove, Christmas Island",mww,5.0,1539459504090
1,green,earthquake,"M 6.7 - 262km NW of Ozernovskiy, Russia","262km NW of Ozernovskiy, Russia",mww,6.7,1539429023560
2,green,earthquake,"M 5.6 - 128km SE of Kimbe, Papua New Guinea","128km SE of Kimbe, Papua New Guinea",mww,5.6,1539312723620
3,green,earthquake,"M 6.5 - 148km S of Severo-Kuril'sk, Russia","148km S of Severo-Kuril'sk, Russia",mww,6.5,1539213362130
4,green,earthquake,"M 6.2 - 94km SW of Kokopo, Papua New Guinea","94km SW of Kokopo, Papua New Guinea",mww,6.2,1539208835130


<hr>
<div>
    <a href="./1-pandas_data_estructura.ipynb">
        <button style="float: left;">&#8592; Notebook Anterior</button>
    </a>
    <a href="./3-creando_dataframes_con_api_requests.ipynb">
        <button style="float: right;">Siguiente Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>